Before running the next block, set up a python3 virtual environemnt. 

First, make sure you're using the correct version of python. In the terminal, type:

>module load python3/3.10.5

Then, create your virtual environment:

>python -m venv .venv

then 

>source .venv/bin/activate

then install the requirements:

>pip install -r requirements.txt

You are now free to run the rest of the code:

In [19]:
#for tokenizer
from sklearn.metrics import mean_squared_error, mean_absolute_error , r2_score
import morfessor
import math
import tokenizers 
from tokenizers import Tokenizer, trainers
from tokenizers.models import BPE
from tokenizers.normalizers import Lowercase
from tokenizers.pre_tokenizers import CharDelimiterSplit
import os
from nltk.tokenize import word_tokenize
import nltk

#for BERT
from transformers import AutoTokenizer,AutoModelForSequenceClassification,TrainingArguments, Trainer, BigBirdTokenizerFast, AutoConfig, BertConfig, BertForSequenceClassification
import torch
from datasets import load_dataset
import numpy


In [2]:
if torch.cuda.is_available():
    print('total gpu:',torch.cuda.get_device_properties(0).total_memory / (1024 ** 3), "GB")
    print('Allocated mem:', torch.cuda.memory_allocated(0) / (1024 ** 3), "GB")
    print('Cache:',  torch.cuda.memory_reserved(0) / (1024 ** 3), "GB")

total gpu: 44.421630859375 GB
Allocated mem: 0.0 GB
Cache: 0.0 GB


# Repression Prediction via LLM
Below are the modules needed to train the LMM. 

In [2]:
def train_tokenizer():
    # We build our custom tokenizer:
    tokenizer = Tokenizer(BPE()) 
    tokenizer.pre_tokenizer = CharDelimiterSplit(' ')

    # We can train this tokenizer by giving it a list of path to text files:
    trainer = trainers.BpeTrainer(show_progress=True)
    tokenizer.train(files=['data/token_data_temp.txt'], trainer=trainer)

    tokenizer.enable_truncation(max_length=512)

    tokenizer.save('./tok/tokenizer.json')

In [3]:
train_tokenizer()

In [22]:
nltk.download('punkt_tab')

input = "CCCAG TCCCT ATTTGAG TTCCAC"

tokens = word_tokenize(input)
print(tokens)

['CCCAG', 'TCCCT', 'ATTTGAG', 'TTCCAC']


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/samuelwu/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
#prepare the data
def get_tokenized_dataset(tokenizer):
    ds = load_dataset('csv', data_files='data/seq_er.csv')
    train_testvalid = ds['train'].train_test_split(test_size=0.2)
    # Split the 10% test + valid in half test, half valid
    test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
    # gather to a single DatasetDict
    ds = {
        'train': train_testvalid['train'],
        'test': test_valid['test'],
        'valid': test_valid['train']}

    def preprocess_function(examples):
        label = examples["Expression Rate"] 
        examples = tokenizer(examples["Token"], truncation=True, padding="max_length", max_length=512)
        
        # Change this to real number
        examples["label"] = float(label)
        return examples

    for split in ds:
        ds[split] = ds[split].map(preprocess_function, remove_columns=["Token", "Expression Rate"])
    return ds

In [23]:
#Choose your tokenizer!

#autotokenizer from DNABERT
tokenizer_dna = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)

#self-made morphessor tokenizer
tokenizer_morph = BigBirdTokenizerFast.from_pretrained("tok", max_len=512)



ds = get_tokenized_dataset(word_tokenize)
print(ds)

Map:   0%|          | 0/72000 [00:00<?, ? examples/s]


TypeError: word_tokenize() got an unexpected keyword argument 'truncation'

In [4]:
LEARNING_RATE = 1e-6
MAX_LENGTH = 256
BATCH_SIZE = 8
EPOCHS = 20

#metrics
def compute_metrics_for_regression(eval_pred):
    logits, labels = eval_pred

    #when training DNABERT you have to use this line
    #logits = logits[0]

    mse = mean_squared_error(labels, logits)
    mae = mean_absolute_error(labels, logits)
    r2 = r2_score(labels, logits)
    single_squared_errors = ((logits - labels).flatten()**2).tolist()
    
    return {"mse": mse, "mae": mae, "r2": r2}

training_args = TrainingArguments(
    output_dir="models/DNABERT_MORPH",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=5,
    load_best_model_at_end=True,
    weight_decay=0.01,
    dataloader_num_workers = 5,
    metric_for_best_model = "mse",
    logging_steps = 10,
)

In [5]:
#Define the model
class RegressionTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0][:, 0]
        loss = torch.nn.functional.mse_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [7]:
# #Train the Model
os.environ["TOKENIZERS_PARALLELISM"] = "true"
# config = BertConfig.from_pretrained("zhihan1996/DNABERT-2-117M")
# model = AutoModelForSequenceClassification.from_pretrained("zhihan1996/DNABERT-2-117M",  config=config, trust_remote_code=True)

config = BertConfig(num_labels=1)
model = BertForSequenceClassification(config)
model.config.num_labels = 1
#load latest checkpoint
model.load_state_dict(torch.load("models/DNABERT_MORPH/checkpoint-22500/pytorch_model.bin"))


trainer = RegressionTrainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["valid"],
    compute_metrics=compute_metrics_for_regression
)
trainer.train()


/restricted/projectnb/cifulab/Sam/LLM-Project/.venv/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


In [40]:
#evaluate the data
trainer.eval_dataset=ds["test"]
trainer.evaluate()

RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/restricted/projectnb/cifulab/Sam/LLM-Project/.venv/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 64, in _worker
    output = module(*input, **kwargs)
  File "/restricted/projectnb/cifulab/Sam/LLM-Project/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "/usr3/graduate/samwu/.cache/huggingface/modules/transformers_modules/DNABERT-2-117M/bert_layers.py", line 859, in forward
    outputs = self.bert(
  File "/restricted/projectnb/cifulab/Sam/LLM-Project/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "/usr3/graduate/samwu/.cache/huggingface/modules/transformers_modules/DNABERT-2-117M/bert_layers.py", line 609, in forward
    encoder_outputs = self.encoder(
  File "/restricted/

In [ ]:
trainer.save_model("final_model/")